In [33]:
import pandas as pd
import re
import numpy as np

from datetime import datetime
import pickle
import os 
import sys
import gzip


from parse_nominations import MyNominationHandler
from parse_ends import MyEndHandler
from parse_revision import MyRevisionsHandler
import process
import config


print(os.getcwd())

# some Ipython 'magic' to reload moduls
%load_ext autoreload

/home/max/pyscripts/wiki_project_local/wiki_project
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Get links to Featured Article Archives
Here we take the html file of the Featured Article Archive page and extract the links to the monthly archives. Subsequently we supply them to wiki.Specail:Export and extract the xml file. It's important  to export templates as well, otherwise the discussions are obmitted.  
* (https://en.wikipedia.org/wiki/Wikipedia:Featured_article_candidates/Featured_log)
* (https://en.wikipedia.org/wiki/Wikipedia:Featured_article_candidates/Archived_nominations)

In [4]:
%autoreload
def get_archives(filename, pattern):
    """ Retrives the links to the monthly FA-Archives"""
    
    # Extract links from Html document
    with open(filename) as file:
        links_to_archive = []
        for line in file:
            matcher = re.search(pattern, line)
            if matcher:
                links_to_archive.append(matcher.group(1))
                    
    
    # Remove links to archives from before 2005 and after 2016
    year_list = ['2003', '2004', '2017', '2018', '2019'] 
    res = []
    for link in links_to_archive:
        if not any([year in link for year in year_list]):
            res.append(link)
    return res

# sucessful nominations
pattern_featured = "(Wikipedia:Featured_article_candidates/Featured_log/.*?)\""
with open('./data/FA_archives.txt', 'w') as f:
    for archive in get_archives('data/FAArchive.html', pattern_featured):
        f.write(archive +'\n')
        
#unsucessful nominations
pattern_candidate = "(Wikipedia:Featured_article_candidates/Archived_nominations/.*?)\""
with open('./data/FAC_archives.txt', 'w') as f:
    for archive in get_archives('data/FACArchive.html', pattern_candidate)[1:]:
        f.write(archive +'\n')

#### Wirte the names of all articles to a file for checking later

In [5]:
FA_archives = './data/FA_all_archives.xml'
FA_articles = './data/FA_all_articles.txt'

def get_articles(in_file, out_file):
    with open(in_file, 'r') as f:
        articles = []
        for line in f:
            match =re.search("Wikipedia:Featured article candidates/(.*)/archive[\\d]", line)
            if match: #None is falsey in Python
                articles.append(match.group(1)) 
    with open(out_file, 'w') as file:
        for art in articles:
            file.write(art +'\n')

            
FAC_archives = './data/FAC_all_archives.xml'
FAC_articles = './log/FAC_all_articles.txt'
#get_articles(FA_archives, FA_articles)
get_articles(FAC_archives, FAC_articles)

FileNotFoundError: [Errno 2] No such file or directory: './data/FAC_all_archives.xml'

### Getting the XML files
The next step is to go to (wiki/Special:Export)[https://en.wikipedia.org/wiki/Special:Export] and use the content of FA(C)_archives to export the current state, including templates of alle Featured Articles. The two resulting files were named FA.xml (for the sucessfull) and FAC.xml (for the unsucessfull ones). 

### Get the nominated articles and time of nomination
MyNominationHandler is in charge of 
1. finding all candidates
2. find the point in time when they were nominated
3. find the last comment (will be used in handling double nominations)

It will produce an .csv file containing all nominations found in the .xml file and write these to ./data/ . A dictionary containing all problematic entries will be stored in ./tmp/ as a pickle object. 

In [6]:
%autoreload
#Parse Sucessful Nominations
nom_handler = MyNominationHandler('./data/FA_nomination.csv', './tmp/FA_prob.pkl')
nom_handler.parse('./data/FA.xml')


index 0 is out of bounds for axis 0 with size 0
Problematic entry in 5114
seems like no title was found.
index 0 is out of bounds for axis 0 with size 0
Problematic entry in 5291
seems like no title was found.
index 0 is out of bounds for axis 0 with size 0
Problematic entry in 6087
seems like no title was found.


In [7]:
%autoreload
# Parse Unsucsessful Nominations
nom_handler = MyNominationHandler('./data/FAC_nomination.csv', './tmp/FAC_prob.pkl')
nom_handler.parse('./data/FAC.xml')


Problem parsing date in 613


### Get all all revisions in which an article was present
To determine the end of the nomination period the last comment is an unreliable indicator. 
Thus we parse the whole revision history of the featured article discussion and store all articles currently nominated. 
Since we need more than the last 1000 revisions, we cannot use wiki/Special:Export. I used [WikiEvent](http://algo.uni-konstanz.de/software/wikievent/). 

In [9]:
%autoreload
end_handler = MyEndHandler('data/FAC_ends.pkl', 'tmp/FAC_ends_prob.pkl')
end_handler.parse('./data/FAChistory.xml')

2004-02-13 19:07:35 is already in the dict. 
 This schould not have happend! (More than once)


### Clean and merge data
#### Sucessfull nominations 

In [14]:
%autoreload
FA_controll = process.merge('./data/FA_nomination.csv', './data/FAC_ends.pkl')
FA_controll = process.decide_end(FA_controll)
FA_controll.drop('dates', 1)
FA_controll.to_csv('./res/FA_merged.csv', sep=';')
#FA_controll = FA_controll[['title', 'start_date', 'date_nomination', 'end_date']]
#FA_controll.to_csv('./data/FA_controll.csv', sep=';')

multiple nominations: 50
number of nominations:                 5280 
artilcles in ends:                10451
unique articles in nominations:       5255 
articles in merged data Frame:            5157
5157 10451


#### Unsucessfull nominations

In [15]:
%autoreload
FAC_controll = process.merge('./data/FAC_nomination.csv', './data/FAC_ends.pkl')
FAC_controll = process.decide_end(FAC_controll)
FAC_controll.drop('dates', 1)
FAC_controll.to_csv('./res/FAC_merged.csv', sep=';')
FAC_controll = FAC_controll[['title', 'start_date', 'date_nomination', 'end_date']]
FAC_controll.to_csv('./data/FAC_controll.csv', sep=';')

multiple nominations: 1845
number of nominations:                 4966 
artilcles in ends:                10451
unique articles in nominations:       3885 
articles in merged data Frame:            3774
3774 10451


In [ ]:
%autoreload
with gzip.open('./data/enwiki_history.xml.gz') as input_file:
    MyRevisionsHandler('./res/revisions_FAC.csv', './data/FAC_controll.csv', True).parse(input_file)

In [ ]:
%autoreload
with gzip.open('./data/enwiki_history.xml.gz') as input_file:
    MyRevisionsHandler('./res/revisions_FA.csv', './data/FA_controll.csv', True).parse(input_file)

In [31]:
FA_controll.loc[FA_controll.title == 'Elvis Presley',]

,idx,title,date_nomination,date_last_comment,has_duplicate,dates,end_date,start_date,nomination_period
2174,3113,Elvis Presley,2008-02-11 22:36:00,2010-02-23 19:28:00,False,"[2006-10-21 19:41:14, 2006-10-21 20:00:36, 200...",2010-02-23 19:28:00,2008-01-28 22:36:00,742 days 20:52:00
